In [1]:
from pathlib import Path
from brain_observatory_qc.pipeline_dev import paired_plane_registration as ppr
import h5py
import numpy as np
import brain_observatory_qc.data_access.from_lims as from_lims
from brain_observatory_qc.pipeline_dev import decrosstalk_roi_timeseries_ica as dc_roi_ts

decrosstalk_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pipeline_validation\decrosstalk')
data_dir = decrosstalk_dir / 'paired_registration_data'

In [3]:
oeid = 1167237081
paired_id = ppr.get_paired_plane_id(oeid)
paired_reg_fn = data_dir / f'{paired_id}_paired_registered.h5'



In [4]:
## To get the whole decrosstalk_data
# It will take > 2x RAM of the h5 file
decrosstalk_data, alpha_list, beta_list = dc_roi_ts.decrosstalk_timeseries_exp(oeid, paired_reg_fn)


dc_fov_dir = data_dir / 'decrosstalk_roi_timeseries'
if not dc_fov_dir.exists():
    dc_fov_dir.mkdir()
decrosstalk_fn = dc_fov_dir / f'{oeid}_decrosstalk.h5'
with h5py.File(decrosstalk_fn, 'w') as f:
    f.create_dataset('data', data=decrosstalk_data)
    f.create_dataset('alpha_list', data=alpha_list)
    f.create_dataset('beta_list', data=beta_list)

# 27 min    

In [ ]:
## Just to get alpha and beta for the experiment:
_, alpha_list, beta_list = dc_roi_ts.decrosstalk_timeseries_exp(oeid, paired_reg_fn, return_recon=False)
alpha = np.mean(alpha_list)
beta = np.mean(beta_list)

## To reduce RAM usage, you can get the decrosstalk_data in chunks:
chunk_size = 5000 # num of frames in each chunk

signal_fn = from_lims.get_motion_corrected_movie_filepath(oeid)
with h5py.File(signal_fn, 'r') as f:
    data_shape = f['data'].shape
data_length = data_shape[0]
num_chunks = int(np.ceil(data_length / chunk_size))
start_frames = np.arange(0, data_length, chunk_size)
end_frames = np.append(start_frames[1:], data_length)

dc_fov_dir = data_dir / 'decrosstalk_roi_timeseries'
if not dc_fov_dir.exists():
    dc_fov_dir.mkdir()
decrosstalk_fn = dc_fov_dir / f'{oeid}_decrosstalk.h5'

i = 0
for start_frame, end_frame in zip(start_frames, end_frames):
    with h5py.File(signal_fn, 'r') as f:
        signal_data = f['data'][start_frame:end_frame]
    with h5py.File(paired_reg_fn, 'r') as f:
        paired_data = f['data'][start_frame:end_frame]
    recon_signal_data = np.zeros_like(signal_data)
    for j in range(signal_data.shape[0]):
        recon_signal_data[j, :, :] = dc_roi_ts.apply_mixing_matrix(alpha, beta, signal_data[j, :, :], paired_data[j, :, :])[0]

    if i == 0:
        with h5py.File(decrosstalk_fn, 'w') as f:
            f.create_dataset('data', data=recon_signal_data, maxshape=(None, data_shape[1], data_shape[2]))
            f.create_dataset('alpha_list', data=alpha_list)
            f.create_dataset('beta_list', data=beta_list)

    else:
        with h5py.File(decrosstalk_fn, 'a') as f:
            f['data'].resize((f['data'].shape[0] + recon_signal_data.shape[0]), axis=0)
            f['data'][start_frame:end_frame] = recon_signal_data
    i += 1